In [1]:
!pip install -q datasets
!pip install pandas
!pip install odfpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 717.0/717.0 kB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for odfpy: filename=odfpy-1.4.1-py2.py3-none-any.whl size=160671 sha256=bf37417a423ae

In [2]:
import pandas as pd

# Load the dataset
file_path = '/content/webdata.ods'  # Replace with your actual file path
df = pd.read_excel(file_path, sheet_name='Sheet1')  # Adjust sheet_name if necessary

# Display the first few rows to verify
print(df.head())


                                                text
0  ### Human: What services does BRC Mind and Bod...
1  ### Human: What services do you offer?### Assi...
2  ### Human: How To Get Our Services?### Assista...
3  ### Human: How TO Get Our membership package?#...
4  ### Human: Where is Our Office?### Assistant: ...


In [3]:
# Define a function to transform the data
def transform_conversation(example):
    conversation_text = example['text']
    segments = conversation_text.split('###')

    reformatted_segments = []

    # Iterate over pairs of segments
    for i in range(1, len(segments), 2):
        human_text = segments[i].strip().replace('Human:', '').strip()

        # Check if there is a corresponding assistant segment before processing
        if i + 1 < len(segments):
            assistant_text = segments[i + 1].strip().replace('Assistant:', '').strip()

            # Apply the new template
            reformatted_segments.append(f'<s>[INST] {human_text} [/INST] {assistant_text} </s>')
        else:
            # Handle the case where there is no corresponding assistant segment
            reformatted_segments.append(f'<s>[INST] {human_text} [/INST] </s>')

    return {'text': ''.join(reformatted_segments)}


In [4]:
# Apply the transformation to each row
df_transformed = df.apply(transform_conversation, axis=1)

# If you want to save the transformed data
df_transformed.to_csv('transformed_dataset.csv', index=False)

# Display the transformed dataset
print(df_transformed.head())


0    {'text': '<s>[INST] What services does BRC Min...
1    {'text': '<s>[INST] What services do you offer...
2    {'text': '<s>[INST] How To Get Our Services? [...
3    {'text': '<s>[INST] How TO Get Our membership ...
4    {'text': '<s>[INST] Where is Our Office? [/INS...
dtype: object
